In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
from io import StringIO
from datetime import timedelta

#prevents warnings from being printed
import warnings
warnings.filterwarnings('ignore')

In [105]:
sd = pd.read_csv('sd_requests.csv')
sd = sd.convert_dtypes()
sd.head()

,id,status,desc,date,depts,docs,poc,msgs
0,15-1810,CLOSED,"Notices of Violation/Notice to Comply, fire in...","December 7, 2015 via web",Code Enforcement,"title,link 5040 ShorehamPlace building permits...",Ginger Rodriguez,"title,item,time ""Request Closed Public"",02. Re..."
1,15-1811,CLOSED,The October 2015 monthly report for SeaWorld,"December 7, 2015 via web",Department of Real Estate and Airport Management,<NA>,Jeffrey Wallace,"title,item,time ""Request Published Public"",,""D..."
2,15-1812,CLOSED,Records related to the following BIDS: Adams ...,"December 7, 2015 via web",City Clerk,"title,link http://www.sandiego.gov/park-and-re...",Mailei Ross-Cerezo,"title,item,time ""Request Closed Public"",Still ..."
3,15-1813,CLOSED,Historical lease payments made by SeaWorld to ...,"December 7, 2015 via web",Department of Real Estate and Airport Management,<NA>,Jeffrey Wallace,"title,item,time ""Request Closed Public"",02. Re..."
4,15-1814,CLOSED,"Open violations, variances, ordinances, approv...","December 7, 2015 via web",Code Enforcement,"title,link Site Plan - 11943 El Camino Real.pd...",Ginger Rodriguez,"title,item,time ""Request Closed Public"",02. Re..."


In [106]:
sd.shape

(28848, 8)

In [107]:
sd = sd.rename(columns = {'date': 'request date'})
sd = sd.fillna('')

sd['city'] = 'San Diego'
sd['state'] = 'CA'

In [136]:
csv_to_df = lambda csv: pd.read_csv(StringIO(csv)) if csv else None
sd['docs_csv'] = sd['docs'].map(csv_to_df)
sd['msgs_csv'] = sd['msgs'].map(csv_to_df)

sd['docs_csv'] = sd['docs_csv'].map(lambda df: df.convert_dtypes().fillna('') if df is not None else None)
sd['msgs_csv'] = sd['msgs_csv'].map(lambda df: df.convert_dtypes().fillna('') if df is not None else None)

In [109]:
sd['num docs'] = sd['docs_csv'].map(lambda df: len(df) if df is not None else 0)
sd.head()

,id,status,desc,request date,depts,docs,poc,msgs,city,state,docs_csv,msgs_csv,num docs
0,15-1810,CLOSED,"Notices of Violation/Notice to Comply, fire in...","December 7, 2015 via web",Code Enforcement,"title,link 5040 ShorehamPlace building permits...",Ginger Rodriguez,"title,item,time ""Request Closed Public"",02. Re...",San Diego,CA,title \ ...,title ...,2
1,15-1811,CLOSED,The October 2015 monthly report for SeaWorld,"December 7, 2015 via web",Department of Real Estate and Airport Management,,Jeffrey Wallace,"title,item,time ""Request Published Public"",,""D...",San Diego,CA,None,title ...,0
2,15-1812,CLOSED,Records related to the following BIDS: Adams ...,"December 7, 2015 via web",City Clerk,"title,link http://www.sandiego.gov/park-and-re...",Mailei Ross-Cerezo,"title,item,time ""Request Closed Public"",Still ...",San Diego,CA,...,title \ 0 R...,1
3,15-1813,CLOSED,Historical lease payments made by SeaWorld to ...,"December 7, 2015 via web",Department of Real Estate and Airport Management,,Jeffrey Wallace,"title,item,time ""Request Closed Public"",02. Re...",San Diego,CA,None,title ...,0
4,15-1814,CLOSED,"Open violations, variances, ordinances, approv...","December 7, 2015 via web",Code Enforcement,"title,link Site Plan - 11943 El Camino Real.pd...",Ginger Rodriguez,"title,item,time ""Request Closed Public"",02. Re...",San Diego,CA,title \ 0 ...,title ...,2


In [110]:
sd['request date'] = sd['request date'].str.extract(r'([A-z][a-z]+\s+\d{1,2},\s+\d{4})')
sd['request date'] = pd.to_datetime(sd['request date'], infer_datetime_format=True)

In [111]:
sd.head()

,id,status,desc,request date,depts,docs,poc,msgs,city,state,docs_csv,msgs_csv,num docs
0,15-1810,CLOSED,"Notices of Violation/Notice to Comply, fire in...",2015-12-07,Code Enforcement,"title,link 5040 ShorehamPlace building permits...",Ginger Rodriguez,"title,item,time ""Request Closed Public"",02. Re...",San Diego,CA,title \ ...,title ...,2
1,15-1811,CLOSED,The October 2015 monthly report for SeaWorld,2015-12-07,Department of Real Estate and Airport Management,,Jeffrey Wallace,"title,item,time ""Request Published Public"",,""D...",San Diego,CA,None,title ...,0
2,15-1812,CLOSED,Records related to the following BIDS: Adams ...,2015-12-07,City Clerk,"title,link http://www.sandiego.gov/park-and-re...",Mailei Ross-Cerezo,"title,item,time ""Request Closed Public"",Still ...",San Diego,CA,...,title \ 0 R...,1
3,15-1813,CLOSED,Historical lease payments made by SeaWorld to ...,2015-12-07,Department of Real Estate and Airport Management,,Jeffrey Wallace,"title,item,time ""Request Closed Public"",02. Re...",San Diego,CA,None,title ...,0
4,15-1814,CLOSED,"Open violations, variances, ordinances, approv...",2015-12-07,Code Enforcement,"title,link Site Plan - 11943 El Camino Real.pd...",Ginger Rodriguez,"title,item,time ""Request Closed Public"",02. Re...",San Diego,CA,title \ 0 ...,title ...,2


In [113]:
#number of open and closed requests for san diego
pd.value_counts(sd['status'])

CLOSED    28843
OPEN          5
Name: status, dtype: Int64

In [125]:
pd.Series(sd['depts'].unique()).to_frame().to_csv('unique sd depts.csv')

In [131]:
dept_count = sd.groupby('depts').count().reset_index()
dept_count = dept_count[['depts', 'id']].rename(columns = {'id': 'requests received'})
dept_count

,depts,requests received
0,,143
1,ADA Compliance & Accessibility,6
2,Airports,41
3,Animal Services,849
4,Assistant Chief Operating Officer,4
...,...,...
469,"Transportation, Performance & Analytics",6
470,"Transportation, Public Records Administration",21
471,"Transportation, Public Records Administration ...",1
472,"Transportation, Stormwater",1


In [135]:
sd.iloc[0,7]

'title,item,time\n"Request Closed\nPublic",02. Released,"December 17, 2015, 4:34pm by Ginger Rodriquez"\n"Document(s) Released\nPublic",5040 ShorehamPlace building permits.pdf,"December 17, 2015, 4:33pm by Ginger Rodriquez"\n"Document(s) Released\nPublic",2015-1810.pdf,"December 15, 2015, 7:54am by Amanda Alvarado, Senior Clerk"\n"Department Assignment\nPublic",Code Enforcement,"December 7, 2015, 5:08pm"\n"Request Opened\nPublic",Request received via web,"December 7, 2015, 5:08pm by Lea Fields-Bernard, Public Records Administration Manager, PRA Program Manager"\n"Request Published\nPublic",,"December 7, 2015, 5:08pm"\n'